In [1]:
from __future__ import division
from statsmodels.stats.multitest import multipletests
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from mlxtend.plotting import plot_sequential_feature_selection as plot_sfs
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from xgboost import XGBClassifier
import pandas as pd
import numpy as np
import re
import pickle
import matplotlib.pyplot as plt
from xgboost import plot_importance
import plotly.graph_objects as go
from tqdm.notebook import tqdm
from sklearn.model_selection import RepeatedKFold
import xgboost as xgb
from sklearn.metrics import roc_auc_score, roc_curve, f1_score, accuracy_score, recall_score, precision_score, confusion_matrix, mean_squared_error
import seaborn as sns
from statannot import add_stat_annotation
from sklearn.linear_model import Lasso, MultiTaskLasso, Ridge, ElasticNet

from sklearn.preprocessing import MinMaxScaler

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression
from sklearn.feature_selection import mutual_info_regression, f_regression

from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RepeatedKFold
from sklearn.decomposition import PCA

from sklearn.utils import shuffle
from sklearn.preprocessing import StandardScaler

from sklearn import metrics


from skopt import BayesSearchCV
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet

import statsmodels.api as sm

import time

import scipy

from sklearn.svm import SVR

from skopt import BayesSearchCV
from sklearn.ensemble import RandomForestRegressor

from statsmodels.stats.multitest import fdrcorrection

from statsmodels.stats import multitest

from sklearn.linear_model import RidgeCV, LassoCV, Ridge, Lasso
from sklearn.ensemble import ExtraTreesClassifier

from collections import Counter
import heapq

import statsmodels.api as sm

from sklearn.ensemble import RandomForestRegressor

import re

from sklearn.preprocessing import MinMaxScaler

import os

In [2]:
frec_df =  pd.read_excel('Data/frequency.xlsx', index_col=0)
frec_df

,Delta_canon_EPP_ORB_left,Delta_canon_EPP_ORB_right,Delta_canon_EPP_IFG_left,Delta_canon_EPP_IFG_right,Delta_canon_EPP_MFG_left,Delta_canon_EPP_MFG_right,Delta_canon_EPP_SFG_left,Delta_canon_EPP_SFG_right,Delta_canon_EPP_HPC_left,Delta_canon_EPP_HPC_right,...,IAF_INS_left,IAF_INS_right,IAF_OCC_left,IAF_OCC_right,Age,Sex,Educ,MMSE,GINI,ODQ
0,0.059161,0.070466,0.051849,0.049289,0.049934,0.047389,0.049015,0.047568,0.041415,0.056212,...,9.0,8.625,8.833333,8.791667,33.0,1.0,19.0,30.0,40.70,99.282421
1,0.054096,0.047097,0.056917,0.044670,0.055201,0.042544,0.048499,0.039344,0.041028,0.037112,...,11.5,11.625,11.375000,11.500000,57.0,2.0,13.0,NaN,40.70,99.739738
2,0.059660,0.051740,0.061596,0.052308,0.061998,0.046384,0.053836,0.056489,0.060980,0.045492,...,10.0,10.375,10.125000,10.166667,32.0,1.0,10.0,NaN,40.70,99.361739
3,0.058654,0.056826,0.060448,0.056367,0.068132,0.053792,0.067543,0.045742,0.058767,0.039325,...,11.0,10.875,11.208333,11.083333,21.0,1.0,13.0,NaN,40.70,99.561273
4,0.043061,0.042042,0.045694,0.039943,0.045708,0.036764,0.039002,0.036255,0.044407,0.037282,...,8.0,9.500,9.625000,8.000000,19.0,1.0,13.0,30.0,40.70,99.822774
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1443,0.035464,0.035469,0.035834,0.034404,0.030824,0.030423,0.031417,0.031326,0.038548,0.033387,...,8.0,10.000,8.000000,9.041667,57.0,2.0,17.0,30.0,44.92,98.650235
1444,0.054815,0.048826,0.049235,0.049774,0.044013,0.046808,0.044252,0.046730,0.053651,0.049409,...,8.5,9.250,8.958333,9.625000,48.0,2.0,17.0,28.0,44.92,99.599221
1445,0.044759,0.052309,0.042457,0.046901,0.038112,0.037699,0.038526,0.039239,0.049349,0.049044,...,9.0,9.000,8.958333,8.000000,57.0,2.0,17.0,29.0,44.92,99.675799
1446,0.059905,0.064894,0.052238,0.051449,0.039586,0.041117,0.042107,0.041310,0.050399,0.052534,...,8.5,8.625,9.916667,9.708333,69.0,2.0,12.0,30.0,44.92,99.641446


## Statistical significance check

In [3]:
outcomes_list = list(frec_df.iloc[:,0:-10])
df_metrics_slopes = pd.DataFrame(columns=outcomes_list)
df_metrics_pval = pd.DataFrame(columns=outcomes_list)
df_metrics_dir = pd.DataFrame(columns=outcomes_list)
df_metrics_slopes

,Delta_canon_EPP_ORB_left,Delta_canon_EPP_ORB_right,Delta_canon_EPP_IFG_left,Delta_canon_EPP_IFG_right,Delta_canon_EPP_MFG_left,Delta_canon_EPP_MFG_right,Delta_canon_EPP_SFG_left,Delta_canon_EPP_SFG_right,Delta_canon_EPP_HPC_left,Delta_canon_EPP_HPC_right,...,IAF_ORB_left,IAF_ORB_right,IAF_IFG_left,IAF_IFG_right,IAF_MFG_left,IAF_MFG_right,IAF_SFG_left,IAF_SFG_right,IAF_HPC_left,IAF_HPC_right


In [4]:
from scipy.stats import linregress

df_metrics_slopes = pd.DataFrame(columns=outcomes_list)

for variable_y in outcomes_list:
    
    c_frec_df_mmse = frec_df[['GINI', variable_y]]
    
    c_frec_df_mmse = c_frec_df_mmse.dropna()
    c_frec_df_mmse.reset_index(drop=True)
    
    y_frec_1A = c_frec_df_mmse[variable_y]
    X_frec_1A = c_frec_df_mmse[['GINI']]
    
    scaler = MinMaxScaler([0.05, 0.95])
    scaling_data = scaler.fit_transform(X_frec_1A)
    X_frec_1A = pd.DataFrame(scaling_data, columns= X_frec_1A.columns, index = X_frec_1A.index)

    X_frec_1A =  X_frec_1A.copy()

    slope, intercept, r_value, p_value, std_err = linregress(X_frec_1A['GINI'], y_frec_1A)
    
    p_value*=df_metrics_slopes.shape[1]
    if(p_value<=0.05):
        df_metrics_slopes.loc['GINI', variable_y] =  str(np.round(slope,3)) + ' (' + str(np.round(p_value,3))+')' 
    
    df_metrics_pval.loc['GINI', variable_y] =  p_value
    df_metrics_dir.loc['GINI', variable_y] =  slope
    
for variable_y in outcomes_list:
    for variable_x in ['Age', 'Educ', 'Sex', 'MMSE']:
    
        c_frec_df_mmse = frec_df[['GINI', variable_x, variable_y]] 

        c_frec_df_mmse = c_frec_df_mmse.dropna()
        c_frec_df_mmse.reset_index(drop=True)
        
        y_frec_1A = c_frec_df_mmse[variable_y]
        X_frec_1A = c_frec_df_mmse[['GINI', variable_x]]
        
        scaler = MinMaxScaler([0.05, 0.95])
        scaling_data = scaler.fit_transform(X_frec_1A)
        X_frec_1A = pd.DataFrame(scaling_data, columns= X_frec_1A.columns, index = X_frec_1A.index)

        X_frec_1A =  X_frec_1A.copy()

        slope, intercept, r_value, p_value, std_err = linregress(X_frec_1A[variable_x],y_frec_1A)
    
        p_value*=df_metrics_slopes.shape[1]
        if(p_value<=0.05):
            df_metrics_slopes.loc[variable_x, variable_y] =  str(np.round(slope,3)) + ' (' + str(np.round(p_value,3))+')' #slope
     
        df_metrics_pval.loc[variable_x, variable_y] =  p_value
        df_metrics_dir.loc[variable_x, variable_y] =  slope

In [5]:
df_metrics_slopes = df_metrics_slopes.T
df_metrics_pval = df_metrics_pval.T
df_metrics_dir = df_metrics_dir.T

In [6]:
df_metrics_GINI = df_metrics_slopes[['GINI']]
df_metrics_GINI = df_metrics_GINI.copy()
df_metrics_GINI.dropna(inplace=True)

In [7]:
outcomes_list = list(df_metrics_GINI.index)

In [8]:
def classifier_ROC(X_, y, fig_show = True, params_ = None):
    
    X_train, X_test, y_train, y_test = train_test_split(X_, y, test_size=0.2, random_state=10000, stratify=y)

    #lista = list(sfs_.k_feature_names_)

    #X_train = X_train[lista]
    #X_test = X_test[lista]

    cv    = RepeatedKFold(n_splits=10, n_repeats=2, random_state=101)
    folds = [(train,test) for train, test in cv.split(X_train, y_train)]
    metrics = ['auc', 'fpr', 'tpr', 'thresholds', 'f1_score', 'accuracy_score', 'recall_score', 'precision_score', 'confusion_matrix', 'model']
    results = {
        'train': {m:[] for m in metrics},
        'val'  : {m:[] for m in metrics},
        'test' : {m:[] for m in metrics}
    }
    X_train = pd.DataFrame(X_train)
    y_train = pd.DataFrame(y_train)


    if(params_!=None):
        params = params_
    else:
        params = {
            'objective'   : 'binary:logistic',
            'eval_metric' : 'logloss'
        }


    plt.rcParams["figure.figsize"] = (10,10)
    dtest = xgb.DMatrix(X_test, label=y_test)
    for train, test in tqdm(folds, total=len(folds)):
        dtrain = xgb.DMatrix(X_train.iloc[train,:], label=y_train.iloc[train])
        dval   = xgb.DMatrix(X_train.iloc[test,:], label=y_train.iloc[test])
        model  = xgb.train(
            dtrain                = dtrain,
            params                = params, 
            evals                 = [(dtrain, 'train'), (dval, 'val')],
            num_boost_round       = 1000,
            verbose_eval          = False,
            early_stopping_rounds = 10,
        )
        sets = [dtrain, dval, dtest]

        for i,ds in enumerate(results.keys()):
            y_preds              = model.predict(sets[i])
            labels               = sets[i].get_label()
            fpr, tpr, thresholds = roc_curve(labels, y_preds)
            results[ds]['fpr'].append(fpr)
            results[ds]['tpr'].append(tpr)
            results[ds]['thresholds'].append(thresholds)
            results[ds]['auc'].append(roc_auc_score(labels, y_preds))

            results[ds]['f1_score'].append(f1_score(labels, np.round(y_preds)))
            results[ds]['accuracy_score'].append(accuracy_score(labels, np.round(y_preds)))
            results[ds]['recall_score'].append(recall_score(labels, np.round(y_preds)))
            results[ds]['precision_score'].append(precision_score(labels, np.round(y_preds)))
            results[ds]['confusion_matrix'].append(confusion_matrix(labels, np.round(y_preds)))
            results[ds]['model'].append(model)


    kind = 'test'
    c_fill      = 'rgba(52, 152, 219, 0.2)'
    c_line      = 'rgba(52, 152, 219, 0.5)'
    c_line_main = 'rgba(41, 128, 185, 1.0)'
    c_grid      = 'rgba(189, 195, 199, 0.5)'
    c_annot     = 'rgba(149, 165, 166, 0.5)'
    c_highlight = 'rgba(192, 57, 43, 1.0)'

    fpr_mean    = np.linspace(0, 1, 100)
    interp_tprs = []

    for i in range(10*2):
        fpr           = results[kind]['fpr'][i]
        tpr           = results[kind]['tpr'][i]
        interp_tpr    = np.interp(fpr_mean, fpr, tpr)
        interp_tpr[0] = 0.0
        interp_tprs.append(interp_tpr)

    tpr_mean     = np.mean(interp_tprs, axis=0)
    tpr_mean[-1] = 1.0
    tpr_std      = 2*np.std(interp_tprs, axis=0)
    tpr_upper    = np.clip(tpr_mean+tpr_std, 0, 1)
    tpr_lower    = tpr_mean-tpr_std
    auc          = np.mean(results[kind]['auc'])
    
    roc_dict = {}
    roc_dict['tpr_mean'] = tpr_mean
    roc_dict['tpr_std'] = tpr_std
    roc_dict['tpr_upper'] = tpr_upper
    roc_dict['tpr_lower'] = tpr_lower
    roc_dict['auc'] = auc

    if(fig_show):
        fig = go.Figure([
            go.Scatter(
                x          = fpr_mean,
                y          = tpr_upper,
                line       = dict(color=c_line, width=1),
                hoverinfo  = "skip",
                showlegend = False,
                name       = 'upper'),
            go.Scatter(
                x          = fpr_mean,
                y          = tpr_lower,
                fill       = 'tonexty',
                fillcolor  = c_fill,
                line       = dict(color=c_line, width=1),
                hoverinfo  = "skip",
                showlegend = False,
                name       = 'lower'),
            go.Scatter(
                x          = fpr_mean,
                y          = tpr_mean,
                line       = dict(color=c_line_main, width=2),
                hoverinfo  = "skip",
                showlegend = True,
                name       = f'AUC: {auc:.3f}')
        ])

        fig.add_shape(
            type ='line', 
            line =dict(dash='dash'),
            x0=0, x1=1, y0=0, y1=1
        )

        fig.update_layout(
            template    = 'plotly_white', 
            title_x     = 0.5,
            xaxis_title = "1 - Specificity",
            yaxis_title = "Sensitivity",
            width       = 800,
            height      = 800,
            legend      = dict(
                yanchor="bottom", 
                xanchor="right", 
                x=0.95,
                y=0.01,
            )
        )

        fig.update_yaxes(
            range       = [0, 1],
            gridcolor   = c_grid,
            scaleanchor = "x", 
            scaleratio  = 1,
            linecolor   = 'black')

        fig.update_xaxes(
            range       = [0, 1],
            gridcolor   = c_grid,
            constrain   = 'domain',
            linecolor   = 'black')



        fig.show()
        
    else:
        print(y.name, 'AUC: ', np.round(auc,2), '+-( ', np.round(np.std(results[kind]['auc']),2), ')', X_.shape)

    return (model, results, np.round(auc,2), roc_dict)

In [9]:
from skopt import BayesSearchCV
def SearchBestModel(case_x, case_y):

    X_train, X_test, y_train, y_test = train_test_split(case_x, case_y, test_size=.2, random_state=1)

    opt_XGB = BayesSearchCV(
            XGBClassifier(),
            {
                'objective'   : ['binary:logistic'],
                'eval_metric' : ['logloss', 'error', 'auc', 'aucpr'],
                'learning_rate': (0.01, 0.1, 0.2),
                'max_depth': (3, 4, 5),
                'subsample': (0.8, 0.9, 1.0),
                'colsample_bytree': (0.8, 0.9, 1.0),
                'reg_alpha': (0, 0.1, 1.0),
                'reg_lambda': (0, 0.1, 1.0),
                #'gamma': (0.001, 0.01, 0.1, 1, 10),
            },
                          
            #n_iter=10,
            #cv=3,
            random_state=1
        )

    opt_XGB.fit(X_train, y_train)


    print('XGB')
    print("val. score: %s" % opt_XGB.best_score_)
    print("test score: %s" % opt_XGB.score(X_test, y_test))
    print("best parameters: %s" % str(opt_XGB.best_params_))
    print('---------------------------------------------\n')
    
    return opt_XGB

## Classification

In [10]:

gini_results_preliminar = {}
gini_results_auc_preliminar = {}
for outcome in outcomes_list:

    frec_df_mmse = frec_df[[outcome, 'GINI']]
    frec_df_mmse = frec_df_mmse.dropna()
    frec_df_mmse.reset_index(drop=True)

   
    mediana = frec_df_mmse[outcome].median()
    frec_df_mmse[outcome + '_b'] = (frec_df_mmse[outcome] > mediana).astype(int)
    
    y_frec_1A = frec_df_mmse[outcome + '_b']
    X_frec_1A = frec_df_mmse[['GINI']]

    scaler = MinMaxScaler([0.05, 0.95])
    scaling_data = scaler.fit_transform(X_frec_1A)
    X_frec_1A_s = pd.DataFrame(scaling_data, columns= X_frec_1A.columns, index = X_frec_1A.index)
    
    X_frec_1A =  X_frec_1A_s.copy()
    
    try:
        r = classifier_ROC(X_frec_1A, y_frec_1A, False)
        gini_results_preliminar[outcome] = r
        gini_results_auc_preliminar[outcome] = r[2]
    except:
        print('Bad: ',outcome + '_b')

  0%|          | 0/20 [00:00<?, ?it/s]

Delta_canon_EPP_ORB_left_b AUC:  0.69 +-(  0.01 ) (1376, 1)


  0%|          | 0/20 [00:00<?, ?it/s]

Delta_canon_EPP_ORB_right_b AUC:  0.68 +-(  0.0 ) (1376, 1)


  0%|          | 0/20 [00:00<?, ?it/s]

...

TF_INS_left_b AUC:  0.67 +-(  0.01 ) (1376, 1)


  0%|          | 0/20 [00:00<?, ?it/s]

TF_INS_right_b AUC:  0.73 +-(  0.01 ) (1376, 1)


  0%|          | 0/20 [00:00<?, ?it/s]

IAF_SFG_right_b AUC:  0.69 +-(  0.0 ) (1376, 1)


In [11]:
top_10_keys_GINI_preliminar = heapq.nlargest(10, gini_results_auc_preliminar, key=gini_results_auc_preliminar.get)
top_10_keys_GINI_preliminar

['Beta1_canon_EPP_HPC_left',
 'Beta1_canon_RPD_HPC_left',
 'Beta1_canon_EPP_HPC_right',
 'Beta1_canon_RPD_HPC_right',
 'Beta1_canon_EPP_CING_left',
 'Beta1_canon_RPD_CING_left',
 'Alpha2_canon_EPP_HPC_left',
 'Alpha2_canon_EPP_HPC_right',
 'Alpha2_canon_RPD_HPC_left',
 'Alpha2_canon_RPD_HPC_right']

In [12]:
gini_results = {}
gini_results_auc = {}
gini_paragram = {}
for outcome in top_10_keys_GINI_preliminar:

    frec_df_mmse = frec_df[[outcome, 'GINI']]
    frec_df_mmse = frec_df_mmse.dropna()
    frec_df_mmse.reset_index(drop=True)

   
    mediana = frec_df_mmse[outcome].median()
    frec_df_mmse[outcome + '_b'] = (frec_df_mmse[outcome] > mediana).astype(int)
    
    y_frec_1A = frec_df_mmse[outcome + '_b']
    X_frec_1A = frec_df_mmse[['GINI']]

    scaler = MinMaxScaler([0.05, 0.95])
    scaling_data = scaler.fit_transform(X_frec_1A)
    X_frec_1A_s = pd.DataFrame(scaling_data, columns= X_frec_1A.columns, index = X_frec_1A.index)
    
    X_frec_1A =  X_frec_1A_s.copy()

    
    opt_XGB = SearchBestModel(X_frec_1A, y_frec_1A)
    params_ = dict(opt_XGB.best_params_)


    r = classifier_ROC(X_frec_1A, y_frec_1A, False, params_)
    gini_results[outcome] = r
    gini_results_auc[outcome] = r[2]
    gini_paragram[outcome] = params_

XGB
val. score: 0.7363636363636363
test score: 0.7463768115942029
best parameters: OrderedDict([('colsample_bytree', 1.0), ('eval_metric', 'auc'), ('learning_rate', 0.2), ('max_depth', 4), ('objective', 'binary:logistic'), ('reg_alpha', 0.1), ('reg_lambda', 0.1), ('subsample', 0.9)])
---------------------------------------------



  0%|          | 0/20 [00:00<?, ?it/s]

Beta1_canon_EPP_HPC_left_b AUC:  0.8 +-(  0.01 ) (1376, 1)
XGB
val. score: 0.7363636363636363
test score: 0.7463768115942029
best parameters: OrderedDict([('colsample_bytree', 1.0), ('eval_metric', 'auc'), ('learning_rate', 0.2), ('max_depth', 4), ('objective', 'binary:logistic'), ('reg_alpha', 0.1), ('reg_lambda', 0.1), ('subsample', 0.9)])
---------------------------------------------



  0%|          | 0/20 [00:00<?, ?it/s]

Beta1_canon_RPD_HPC_left_b AUC:  0.8 +-(  0.01 ) (1376, 1)
XGB
val. score: 0.7309090909090908
test score: 0.7753623188405797
best parameters: OrderedDict([('colsample_bytree', 0.9), ('eval_metric', 'logloss'), ('learning_rate', 0.01), ('max_depth', 5), ('objective', 'binary:logistic'), ('reg_alpha', 1.0), ('reg_lambda', 0.1), ('subsample', 0.8)])
---------------------------------------------



...

Alpha2_canon_EPP_HPC_right_b AUC:  0.78 +-(  0.0 ) (1376, 1)
XGB
val. score: 0.7563636363636363
test score: 0.7391304347826086
best parameters: OrderedDict([('colsample_bytree', 0.9), ('eval_metric', 'error'), ('learning_rate', 0.01), ('max_depth', 3), ('objective', 'binary:logistic'), ('reg_alpha', 0.1), ('reg_lambda', 1.0), ('subsample', 0.8)])
---------------------------------------------



  0%|          | 0/20 [00:00<?, ?it/s]

Alpha2_canon_RPD_HPC_left_b AUC:  0.77 +-(  0.01 ) (1376, 1)
XGB
val. score: 0.7318181818181818
test score: 0.7536231884057971
best parameters: OrderedDict([('colsample_bytree', 1.0), ('eval_metric', 'logloss'), ('learning_rate', 0.01), ('max_depth', 4), ('objective', 'binary:logistic'), ('reg_alpha', 0.1), ('reg_lambda', 1.0), ('subsample', 0.8)])
---------------------------------------------



  0%|          | 0/20 [00:00<?, ?it/s]

Alpha2_canon_RPD_HPC_right_b AUC:  0.78 +-(  0.0 ) (1376, 1)


In [13]:
top_10_keys_GINI = heapq.nlargest(10, gini_results_auc, key=gini_results_auc.get)
top_10_keys_GINI

['Beta1_canon_EPP_HPC_left',
 'Beta1_canon_RPD_HPC_left',
 'Beta1_canon_EPP_HPC_right',
 'Beta1_canon_RPD_HPC_right',
 'Beta1_canon_EPP_CING_left',
 'Beta1_canon_RPD_CING_left',
 'Alpha2_canon_EPP_HPC_right',
 'Alpha2_canon_RPD_HPC_right',
 'Alpha2_canon_EPP_HPC_left',
 'Alpha2_canon_RPD_HPC_left']

### Classification:  Gini + Age

In [14]:

gini_age_results_preliminar = {}
gini_age_results_auc_preliminar = {}
for outcome in outcomes_list:

    frec_df_mmse = frec_df[[outcome, 'GINI', 'Age']]
    frec_df_mmse = frec_df_mmse.dropna()
    frec_df_mmse.reset_index(drop=True)

   
    mediana = frec_df_mmse[outcome].median()
    frec_df_mmse[outcome + '_b'] = (frec_df_mmse[outcome] > mediana).astype(int)
    
    y_frec_1A = frec_df_mmse[outcome + '_b']
    X_frec_1A = frec_df_mmse[['GINI', 'Age']]

    scaler = MinMaxScaler([0.05, 0.95])
    scaling_data = scaler.fit_transform(X_frec_1A)
    X_frec_1A_s = pd.DataFrame(scaling_data, columns= X_frec_1A.columns, index = X_frec_1A.index)
    
    X_frec_1A =  X_frec_1A_s.copy()
    
    try:
        r = classifier_ROC(X_frec_1A, y_frec_1A, False)
        gini_age_results_preliminar[outcome] = r
        gini_age_results_auc_preliminar[outcome] = r[2]
    except:
        print('Bad: ',outcome + '_b')

  0%|          | 0/20 [00:00<?, ?it/s]

Delta_canon_EPP_ORB_left_b AUC:  0.68 +-(  0.01 ) (1376, 2)


  0%|          | 0/20 [00:00<?, ?it/s]

Delta_canon_EPP_ORB_right_b AUC:  0.65 +-(  0.01 ) (1376, 2)


  0%|          | 0/20 [00:00<?, ?it/s]

...

TF_INS_left_b AUC:  0.62 +-(  0.01 ) (1376, 2)


  0%|          | 0/20 [00:00<?, ?it/s]

TF_INS_right_b AUC:  0.66 +-(  0.01 ) (1376, 2)


  0%|          | 0/20 [00:00<?, ?it/s]

IAF_SFG_right_b AUC:  0.65 +-(  0.01 ) (1376, 2)


In [15]:
top_10_keys_GINI_AGE_preliminar = heapq.nlargest(10, gini_age_results_auc_preliminar, key=gini_age_results_auc_preliminar.get)

In [ ]:
gini_age_results = {}
gini_age_results_auc = {}
gini_age_paragram = {}
for outcome in top_10_keys_GINI_AGE_preliminar:

    frec_df_mmse = frec_df[[outcome, 'GINI', 'Age']]
    frec_df_mmse = frec_df_mmse.dropna()
    frec_df_mmse.reset_index(drop=True)

   
    mediana = frec_df_mmse[outcome].median()
    frec_df_mmse[outcome + '_b'] = (frec_df_mmse[outcome] > mediana).astype(int)
    
    y_frec_1A = frec_df_mmse[outcome + '_b']
    X_frec_1A = frec_df_mmse[['GINI', 'Age']]

    scaler = MinMaxScaler([0.05, 0.95])
    scaling_data = scaler.fit_transform(X_frec_1A)
    X_frec_1A_s = pd.DataFrame(scaling_data, columns= X_frec_1A.columns, index = X_frec_1A.index)
    
    X_frec_1A =  X_frec_1A_s.copy()

    
    opt_XGB = SearchBestModel(X_frec_1A, y_frec_1A)
    params_ = dict(opt_XGB.best_params_)


    r = classifier_ROC(X_frec_1A, y_frec_1A, False, params_)
    gini_age_results[outcome] = r
    gini_age_results_auc[outcome] = r[2]
    gini_age_paragram[outcome] = params_

XGB
val. score: 0.7327272727272728
test score: 0.7246376811594203
best parameters: OrderedDict([('colsample_bytree', 0.9), ('eval_metric', 'aucpr'), ('learning_rate', 0.01), ('max_depth', 3), ('objective', 'binary:logistic'), ('reg_alpha', 0.0), ('reg_lambda', 0.1), ('subsample', 0.9)])
---------------------------------------------



  0%|          | 0/20 [00:00<?, ?it/s]

Low_subj_spec_EPP_OCC_right_b AUC:  0.79 +-(  0.0 ) (1376, 2)


In [ ]:
top_10_keys_GINI_age = heapq.nlargest(10, gini_age_results_auc, key=gini_age_results_auc.get)
top_10_keys_GINI_age

### Classification:  Gini + Educ

In [ ]:

gini_educ_results_preliminar = {}
gini_educ_results_auc_preliminar = {}
for outcome in outcomes_list:

    frec_df_mmse = frec_df[[outcome, 'GINI', 'Educ']]
    frec_df_mmse = frec_df_mmse.dropna()
    frec_df_mmse.reset_index(drop=True)

   
    mediana = frec_df_mmse[outcome].median()
    frec_df_mmse[outcome + '_b'] = (frec_df_mmse[outcome] > mediana).astype(int)
    
    y_frec_1A = frec_df_mmse[outcome + '_b']
    X_frec_1A = frec_df_mmse[['GINI', 'Educ']]

    scaler = MinMaxScaler([0.05, 0.95])
    scaling_data = scaler.fit_transform(X_frec_1A)
    X_frec_1A_s = pd.DataFrame(scaling_data, columns= X_frec_1A.columns, index = X_frec_1A.index)
    
    X_frec_1A =  X_frec_1A_s.copy()
    
    try:
        r = classifier_ROC(X_frec_1A, y_frec_1A, False)
        gini_educ_results_preliminar[outcome] = r
        gini_educ_results_auc_preliminar[outcome] = r[2]
    except:
        print('Bad: ',outcome + '_b')

In [ ]:
top_10_keys_GINI_educ_preliminar = heapq.nlargest(10, gini_educ_results_auc_preliminar, key=gini_educ_results_auc_preliminar.get)

In [ ]:
gini_educ_results = {}
gini_educ_results_auc = {}
gini_educ_paragram = {}
for outcome in top_10_keys_GINI_educ_preliminar:

    frec_df_mmse = frec_df[[outcome, 'GINI', 'Educ']]
    frec_df_mmse = frec_df_mmse.dropna()
    frec_df_mmse.reset_index(drop=True)

   
    mediana = frec_df_mmse[outcome].median()
    frec_df_mmse[outcome + '_b'] = (frec_df_mmse[outcome] > mediana).astype(int)
    
    y_frec_1A = frec_df_mmse[outcome + '_b']
    X_frec_1A = frec_df_mmse[['GINI', 'Educ']]

    scaler = MinMaxScaler([0.05, 0.95])
    scaling_data = scaler.fit_transform(X_frec_1A)
    X_frec_1A_s = pd.DataFrame(scaling_data, columns= X_frec_1A.columns, index = X_frec_1A.index)
    
    X_frec_1A =  X_frec_1A_s.copy()

    
    opt_XGB = SearchBestModel(X_frec_1A, y_frec_1A)
    params_ = dict(opt_XGB.best_params_)


    r = classifier_ROC(X_frec_1A, y_frec_1A, False, params_)
    gini_educ_results[outcome] = r
    gini_educ_results_auc[outcome] = r[2]
    gini_educ_paragram[outcome] = params_

In [ ]:
top_10_keys_gini_educ = heapq.nlargest(10, gini_educ_results_auc, key=gini_educ_results_auc.get)
top_10_keys_gini_educ

### Classification:  Gini + Sex

In [ ]:

gini_sex_results_preliminar = {}
gini_sex_results_auc_preliminar = {}
for outcome in outcomes_list:

    frec_df_mmse = frec_df[[outcome, 'GINI', 'Sex']]
    frec_df_mmse = frec_df_mmse.dropna()
    frec_df_mmse.reset_index(drop=True)

   
    mediana = frec_df_mmse[outcome].median()
    frec_df_mmse[outcome + '_b'] = (frec_df_mmse[outcome] > mediana).astype(int)
    
    y_frec_1A = frec_df_mmse[outcome + '_b']
    X_frec_1A = frec_df_mmse[['GINI', 'Sex']]

    scaler = MinMaxScaler([0.05, 0.95])
    scaling_data = scaler.fit_transform(X_frec_1A)
    X_frec_1A_s = pd.DataFrame(scaling_data, columns= X_frec_1A.columns, index = X_frec_1A.index)
    
    X_frec_1A =  X_frec_1A_s.copy()
    
    try:
        r = classifier_ROC(X_frec_1A, y_frec_1A, False)
        gini_sex_results_preliminar[outcome] = r
        gini_sex_results_auc_preliminar[outcome] = r[2]
    except:
        print('Bad: ',outcome + '_b')

In [ ]:
top_10_keys_GINI_sex_preliminar = heapq.nlargest(10, gini_sex_results_auc_preliminar, key=gini_sex_results_auc_preliminar.get)

In [ ]:
gini_sex_results = {}
gini_sex_results_auc = {}
gini_sex_paragram = {}
for outcome in top_10_keys_GINI_sex_preliminar:

    frec_df_mmse = frec_df[[outcome, 'GINI', 'Sex']]
    frec_df_mmse = frec_df_mmse.dropna()
    frec_df_mmse.reset_index(drop=True)

   
    mediana = frec_df_mmse[outcome].median()
    frec_df_mmse[outcome + '_b'] = (frec_df_mmse[outcome] > mediana).astype(int)
    
    y_frec_1A = frec_df_mmse[outcome + '_b']
    X_frec_1A = frec_df_mmse[['GINI', 'Sex']]

    scaler = MinMaxScaler([0.05, 0.95])
    scaling_data = scaler.fit_transform(X_frec_1A)
    X_frec_1A_s = pd.DataFrame(scaling_data, columns= X_frec_1A.columns, index = X_frec_1A.index)
    
    X_frec_1A =  X_frec_1A_s.copy()

    
    opt_XGB = SearchBestModel(X_frec_1A, y_frec_1A)
    params_ = dict(opt_XGB.best_params_)
    

    r = classifier_ROC(X_frec_1A, y_frec_1A, False, params_)
    gini_sex_results[outcome] = r
    gini_sex_results_auc[outcome] = r[2]
    gini_sex_paragram[outcome] = params_

In [ ]:
top_10_keys_gini_sex = heapq.nlargest(10, gini_sex_results_auc, key=gini_sex_results_auc.get)
top_10_keys_gini_sex

### Classification:  Gini + Cognition

In [ ]:

gini_mmse_results_preliminar = {}
gini_mmse_results_auc_preliminar = {}
for outcome in outcomes_list:

    frec_df_mmse = frec_df[[outcome, 'GINI', 'MMSE']]
    frec_df_mmse = frec_df_mmse[frec_df_mmse['MMSE'] >= 21]
    frec_df_mmse = frec_df_mmse.dropna()
    frec_df_mmse.reset_index(drop=True)

   
    mediana = frec_df_mmse[outcome].median()
    frec_df_mmse[outcome + '_b'] = (frec_df_mmse[outcome] > mediana).astype(int)
    
    y_frec_1A = frec_df_mmse[outcome + '_b']
    X_frec_1A = frec_df_mmse[['GINI', 'MMSE']]

    scaler = MinMaxScaler([0.05, 0.95])
    scaling_data = scaler.fit_transform(X_frec_1A)
    X_frec_1A_s = pd.DataFrame(scaling_data, columns= X_frec_1A.columns, index = X_frec_1A.index)
    
    X_frec_1A =  X_frec_1A_s.copy()
    
    try:
        r = classifier_ROC(X_frec_1A, y_frec_1A, False)
        gini_mmse_results_preliminar[outcome] = r
        gini_mmse_results_auc_preliminar[outcome] = r[2]
    except:
        print('Bad: ',outcome + '_b')

In [ ]:
top_10_keys_GINI_mmse_preliminar = heapq.nlargest(10, gini_mmse_results_auc_preliminar, key=gini_mmse_results_auc_preliminar.get)

In [ ]:
gini_mmse_results = {}
gini_mmse_results_auc = {}
gini_mmse_paragram = {}
for outcome in top_10_keys_GINI_mmse_preliminar:

    frec_df_mmse = frec_df[[outcome, 'GINI', 'MMSE']]
    frec_df_mmse = frec_df_mmse[frec_df_mmse['MMSE'] >= 21]
    frec_df_mmse = frec_df_mmse.dropna()
    frec_df_mmse.reset_index(drop=True)

   
    mediana = frec_df_mmse[outcome].median()
    frec_df_mmse[outcome + '_b'] = (frec_df_mmse[outcome] > mediana).astype(int)
    
    y_frec_1A = frec_df_mmse[outcome + '_b']
    X_frec_1A = frec_df_mmse[['GINI', 'MMSE']]

    scaler = MinMaxScaler([0.05, 0.95])
    scaling_data = scaler.fit_transform(X_frec_1A)
    X_frec_1A_s = pd.DataFrame(scaling_data, columns= X_frec_1A.columns, index = X_frec_1A.index)
    
    X_frec_1A =  X_frec_1A_s.copy()

    
    opt_XGB = SearchBestModel(X_frec_1A, y_frec_1A)
    params_ = dict(opt_XGB.best_params_)
    

    r = classifier_ROC(X_frec_1A, y_frec_1A, False, params_)
    gini_mmse_results[outcome] = r
    gini_mmse_results_auc[outcome] = r[2]
    gini_mmse_paragram[outcome] = params_

In [ ]:
top_10_keys_gini_mmse = heapq.nlargest(10, gini_mmse_results_auc, key=gini_mmse_results_auc.get)
top_10_keys_gini_mmse